In [108]:
import urllib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF
import pandas as pd
import numpy as np
import umap
import itertools

# Bokeh
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.models import HoverTool, CustomJS, ColumnDataSource, Slider, Range1d
from bokeh.layouts import column
from bokeh.palettes import all_palettes

output_notebook()

Loading BokehJS ...

In [109]:
# define Enrichr library names here if you're using Enrichr libraries
all_libraries = ['ChEA_2016']

In [110]:
# open Enrichr library from online
def get_Enrichr_library(library_index):
    # processes library data
    raw_library_data = []
    library_data = []

    # open Enrichr library from online
    with urllib.request.urlopen('https://amp.pharm.mssm.edu/Enrichr/geneSetLibrary?mode=text&libraryName=' + all_libraries[library_index]) as f:
        for line in f.readlines():
                raw_library_data.append(line.decode("utf-8").split("\t\t"))

    #OR
    #locally upload library
    '''
    with open('filepath', 'r') as f:
        for line in f.readlines():
           raw_library_data.append(line.split("\t\t"))
    '''

    name = []
    gene_list = []

    for i in range(len(raw_library_data)):
        name += [raw_library_data[i][0]]
        raw_genes = raw_library_data[i][1].replace('\t', ' ')
        gene_list += [raw_genes[:-1]]

    library_data = [list(a) for a in zip(name, gene_list)]
    
    return library_data

In [111]:
library_data = get_Enrichr_library(0)

df = pd.DataFrame(data = library_data, columns = ['Name', 'Genes'])

gene_list = df['Genes']

tfidf_vectorizer = TfidfVectorizer(
    min_df=1, # says it will ignore genes that fall in under 1 gene set
    max_df=0.85, # says it will ignore genes that fall in over 85% of the gene sets
    max_features = 10000, # I think this means it only looks at 10000 genes but I'm not totally sure
    ngram_range=(1, 1) # says it will only look at individual genes (because order doesn't matter in our case)
)

tfidf = tfidf_vectorizer.fit_transform(gene_list)

# Save the feature names for later to create topic summaries
tfidf_fn = tfidf_vectorizer.get_feature_names()

# plot using UMAP after tfidf
reduce = umap.UMAP()
reducer.fit(tfidf)
embedding = reducer.transform(tfidf)

embedding = pd.DataFrame(embedding, columns=['x','y'])

source1 = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            alpha = [0.7] * embedding.shape[0],
            size = [7] * embedding.shape[0], # can make size of the circles smaller by reducing the 7 here
            gene_set = df['Name']
        )
    )

# just a print check here to make sure it's working correctly (feel free to delete)
print('There are ', embedding.shape[0], ' gene sets in this visualization')

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
plot_emb.circle('x', 'y', size='size', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source1, name="df")

show(plot_emb)

There are  645  gene sets in this visualization


In [112]:
# Create and run NMF model

n_comp = 6 # this is the number of groups you are splitting your gene set into

nmf = NMF(
    n_components=n_comp, 
    max_iter=100000,
    alpha=0.0
)

W = nmf.fit_transform(tfidf)
H = nmf.components_
nmf_embedding = nmf.transform(tfidf)

In [113]:
reducer = umap.UMAP()
reducer.fit(W)
embedding = reducer.transform(W)

embedding = umap.UMAP().fit_transform(tfidf.todense())
embedding = pd.DataFrame(embedding, columns=['x','y'])

# this will color by the groups created during NMF
# it's useful to look at what the model considers to be a "group"
# can use it help adjust the number of components in the NMF model
embedding['hue'] = nmf_embedding.argmax(axis = 1)
my_colors = [all_palettes['Category20'][20][i] for i in embedding.hue]

source = ColumnDataSource(
        data=dict(
            x = embedding.x,
            y = embedding.y,
            alpha = [0.7] * embedding.shape[0],
            size = [7] * embedding.shape[0],
            gene_set = df['Name'],
            colors = my_colors
        )
    )

print(embedding.shape[0])

hover_emb = HoverTool(names=["df"], tooltips="""
    <div style="margin: 10">
        <div style="margin: 0 auto; width:300px;">
            <span style="font-size: 12px; font-weight: bold;">Gene Set:</span>
            <span style="font-size: 12px">@gene_set</span>
        </div>
    </div>
    """)
tools_emb = [hover_emb, 'pan', 'wheel_zoom', 'reset', 'save']
plot_emb = figure(plot_width=700, plot_height=700, tools=tools_emb)
plot_emb.circle('x', 'y', size='size', 
                 alpha='alpha', line_alpha=0, line_width=0.01, source=source, fill_color='colors', name="df")

show(plot_emb)

645


In [116]:
# this will print out the most common genes in each group
# not super useful but kind of interesting
n_topics = n_comp
n_top_words = 15

print("Topics found via NMF:")
for topic_idx, topic in enumerate(nmf.components_):
    print("\nTopic {}:".format(topic_idx+1))
    print(" ".join(['[{}]'.format(tfidf_fn[i]) for i in topic.argsort()[:-n_top_words - 1:-1]]))
print()

Topics found via NMF:

Topic 1:
[h2] [hla] [sep] [mar] [snar] [pisd] [ps1] [trib1] [hes1] [nkx2] [klf6] [cited2] [tle3] [btg1] [dusp6]

Topic 2:
[c4bp] [epb4] [ps1] [mir181b] [4921521f21rik] [pinc] [mir181a] [hsfy2] [2900092d14rik] [aox4] [1700029f09rik] [1500015o10rik] [mtap2] [1700066m21rik] [9430031j16rik]

Topic 3:
[nkx2] [h2] [nkx6] [zic1] [foxf1a] [alx3] [foxf2] [hmx1] [dmrt3] [hoxb3] [vgll2] [tlx1] [kcna1] [phox2b] [wnt1]

Topic 4:
[rcc1] [snhg3] [c1orf93] [tprg1l] [cdk11a] [kiaa0754] [loc100128003] [loc100133612] [znf362] [plekhn1] [dvl1] [c1orf113] [mst1p2] [c1orf86] [miip]

Topic 5:
[mir] [mmu] [hsa] [gbp5] [gclc] [gck] [gch1] [gcc2] [gbx2] [gbx1] [gbp6] [gbp2] [gcm1] [gbp1] [gbf1]

Topic 6:
[rnf12] [phc1] [pou5f1] [porcn] [mapk1ip1] [zfp57] [lrrc2] [tdgf1] [cd38] [ifitm1] [2310003c23rik] [ephx2] [spp1] [rif1] [dppa3]

